In [7]:
import pandas as pd
from safetensors.torch import save_model

# read and deal the wrong data lines
data = pd.read_csv('../data/Sample_Superstore.csv', encoding='utf-8', on_bad_lines='skip')
# data = pd.read_csv('../data/Sample_Superstore.csv', encoding='utf-8')
# data.head()

#extract the sub data of Columns: Row ID+Category+Sub-Category+Product Name
data_classdification = data[['Row ID', 'Category', 'Sub-Category', 'Product Name']]
data_classdification.head()

# save the data
data_classdification.to_csv('../data/classification_data.csv', index=False, encoding='utf-8')

In [67]:
# use bert from huggingface to train a bet model to classify the data's category and sub-category
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Check if GPU is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the data
data = pd.read_csv('../data/classification_data.csv')

# Encode the labels
label_encoder = LabelEncoder()
# data['Category'] = label_encoder.fit_transform(data['Category'])
# data['Sub-Category'] = label_encoder.fit_transform(data['Sub-Category'])

# Combine Category and Sub-Category into a single label
data['Label'] = data['Category'].astype(str) + '-' + data['Sub-Category'].astype(str)
data['Label'] = label_encoder.fit_transform(data['Label'])

# Split the data into training, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['Product Name'], data['Label'], test_size=0.3)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5)

# Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

class ClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        # labels is a pandas series, convert it to a list
        self.labels = labels.tolist()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # print(idx)
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ClassificationDataset(train_encodings, train_labels)
val_dataset = ClassificationDataset(val_encodings, val_labels)
test_dataset = ClassificationDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Move data to device
for batch in train_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
for batch in val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}


D:\applications\anaconda\envs\hku_nlp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [62]:
train_dataset[0]

0


{'input_ids': tensor([  101, 12140, 22207, 23951,  2072,  1011,  3193,  4472,  2005,  8031,
          3001,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [65]:
train_labels.values[0]

6

In [70]:
# mapping function of label num to label text
def map_num_to_label(num):
    combine = label_encoder.inverse_transform([num])[0]
    category = combine.split('-')[0]
    sub_category = combine.split('-')[1]
    # decoder again
    # category = label_encoder.inverse_transform([int(category)])
    # sub_category = label_encoder.inverse_transform([int(sub_category)])
    return category, sub_category
# test the mapping function
print(map_num_to_label(6))
# build the dictionary of label num to label text
label_num_to_text = {}
for i in range(len(label_encoder.classes_)):
    label_num_to_text[i] = map_num_to_label(i)
print(label_num_to_text)

('Office Supplies', 'Binders')
{0: ('Furniture', 'Bookcases'), 1: ('Furniture', 'Chairs'), 2: ('Furniture', 'Furnishings'), 3: ('Furniture', 'Tables'), 4: ('Office Supplies', 'Appliances'), 5: ('Office Supplies', 'Art'), 6: ('Office Supplies', 'Binders'), 7: ('Office Supplies', 'Envelopes'), 8: ('Office Supplies', 'Fasteners'), 9: ('Office Supplies', 'Labels'), 10: ('Office Supplies', 'Paper'), 11: ('Office Supplies', 'Storage'), 12: ('Office Supplies', 'Supplies'), 13: ('Technology', 'Accessories'), 14: ('Technology', 'Copiers'), 15: ('Technology', 'Machines'), 16: ('Technology', 'Phones')}


In [68]:
# Load the model, save the pretrained model at ../models/bert-base-uncased
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_), cache_dir='../models/bert-base-uncased')
model.to(device)  # Move model to GPU if available

# Define training arguments, save checkpoints at each epoch to the specified directory
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model on the test set
trainer.evaluate(test_dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
D:\applications\anaconda\envs\hku_nlp\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.136400,0.114504
2,0.028300,0.026235
3,0.002700,0.016300


{'eval_loss': 0.01274411752820015,
 'eval_runtime': 6.6297,
 'eval_samples_per_second': 223.991,
 'eval_steps_per_second': 14.028,
 'epoch': 3.0}

In [69]:
from sklearn.metrics import accuracy_score

# Get predictions on the test set
predictions = trainer.predict(test_dataset)

# Get the predicted labels
preds = predictions.predictions.argmax(-1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, preds)
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.9973


In [74]:
# predict the word "Comfortable Executive Mouse"
inputs = tokenizer("Comfortable Executive Mouse", return_tensors="pt").to(device)
outputs = model(**inputs)
preds = outputs.logits.argmax(-1)
print(label_num_to_text[preds.item()])

('Technology', 'Accessories')
